In [1]:
using Pipe
using Iterators
using DataStructures

In [2]:
function loadphrases(filename)
    (@pipe readall(filename) |> split(_,'\n'))[1:end-1];
end

function load_phrases_and_embeddings(base_filename)
    phrases = loadphrases(base_filename * ".phrases.txt")
    ps = readcsv(base_filename * ".csv")
    @assert size(ps,1)==length(phrases)
    phrases,ps
end

load_phrases_and_embeddings (generic function with 1 method)

In [3]:
#phrase_range = 14380:16000  #old testerment14380:40069
#ps = readcsv("results/gutten.csv")[phrase_range,:]
#phrases = loadphrases("results/gutten.phrases.txt") [phrase_range]

In [59]:
phrases,ps = load_phrases_and_embeddings("results/wikipedia_turtle")

(SubString{ASCIIString}["turtles are *UNKNOWN* of the order *UNKNOWN* *UNKNOWN* or *UNKNOWN*","turtle , *UNKNOWN* or *UNKNOWN*","the word *UNKNOWN* is popular among *UNKNOWN* , scientists , and conservationists working with these animals as a catch-all name for any member of the *UNKNOWN* *UNKNOWN* , which includes all turtles living and *UNKNOWN* , as well as their immediate ancestors .","anatomy and *UNKNOWN* largest living *UNKNOWN* is the *UNKNOWN* sea turtle *UNKNOWN* *UNKNOWN* *UNKNOWN* *UNKNOWN* , which reaches a shell length of","giant *UNKNOWN* of the *UNKNOWN* *UNKNOWN* , *UNKNOWN* , and others were relatively widely distributed around the world into prehistoric times , and are known to have existed in north and south *UNKNOWN* , *UNKNOWN* , and *UNKNOWN* .","they became *UNKNOWN* at the same time as the appearance of man , and it is assumed humans hunted them for food .","the only surviving giant *UNKNOWN* are on the *UNKNOWN* and *UNKNOWN* islands , and can grow to over","n

In [60]:
using PyCall
@pyimport sklearn
@pyimport sklearn.mixture as sklm
@pyimport sklearn.cluster as skl_cluster

In [259]:
function show_clusters(words, labels)
    clusters_inds = DefaultDict(Int, Vector{Int64}, ()->Int64[])
    for ii in 1:size(ps,1)
        push!(clusters_inds[labels[ii]], ii)
    end
    clusters_inds
    
    for ii in keys(clusters_inds)
        println("** $ii **")
        for word in words[clusters_inds[ii]]
            println(" - ", word)
        end
        println()
    end
    clusters_inds
end


show_clusters (generic function with 2 methods)

In [62]:
using Distances
push!(LOAD_PATH, "../word-embeddings2")
import WordEmbeddings.cosine_sim

In [82]:
pps = ps'
pps_cosine_sim = Number[WordEmbeddings.cosine_sim(pps[:,ii],pps[:,jj]) for ii in 1:size(pps,2), jj in 1:size(pps,2)];
pps_manhatten_sim = - pairwise(Cityblock(), pps);
pps_euclidean_sim = -pairwise(Euclidean(),pps);


In [243]:
affinity = pycall(pybuiltin("intern"),PyObject, "precomputed")
affprop = skl_cluster.AffinityPropagation(damping=0.9,affinity=affinity)

affprop[:fit_predict](pps_cosine_sim)
centers = affprop[:cluster_centers_indices_] + 1 #julia indexing
for phrase in phrases[centers]
    println(" - ", phrase)
end

 - turtle , *UNKNOWN* or *UNKNOWN*
 - turtles are divided into two groups according to how they withdraw their necks into their shells *UNKNOWN* something the ancestral *UNKNOWN* could not do *UNKNOWN* .
 - *UNKNOWN* turtles usually have *UNKNOWN* ridges for slicing through their prey .
 - the inner layer of a turtle 's shell is made up of about 60 bones that include portions of the backbone and the ribs , meaning the turtle can not crawl out of its shell .
 - instead , they breathe in two ways .
 - most *UNKNOWN* have a large , *UNKNOWN* shell that makes it difficult for predators to crush the shell between their *UNKNOWN* .
 - in some species , shells may have red , orange , yellow , or gray markings , often spots , lines , or irregular blotches .
 - to endure through *UNKNOWN* conditions , such as winter periods trapped beneath ice or within *UNKNOWN* mud at the bottom of ponds , turtles utilize two general physiological mechanisms .
 - *UNKNOWN* are famous for moving slowly , in pa

In [260]:
clusters = show_clusters(phrases, affprop[:labels_])

** 2 **
 - instead of having teeth , which they appear to have lost about *UNKNOWN* million years ago , long in the tooth : *UNKNOWN* proves turtles evolve ... very slowly the upper and lower *UNKNOWN* of the turtle are covered by *UNKNOWN* ridges .
 - *UNKNOWN* turtles usually have *UNKNOWN* ridges for slicing through their prey .
 - *UNKNOWN* turtles have *UNKNOWN* ridges that help them cut through tough plants .
 - the shells of *UNKNOWN* sea turtles are extremely light because they lack *UNKNOWN* and contain many *UNKNOWN* . it has been suggested by *UNKNOWN* *UNKNOWN* 2002 *UNKNOWN* that the turtle shell can function as *UNKNOWN* buffer .
 - amphibious turtles normally have limbs similar to those of *UNKNOWN* , except the feet are *UNKNOWN* and often have long claws .
 - in captivity
 - *UNKNOWN* species are the most endangered , closely followed by the five endemic species from *UNKNOWN* .
 - *UNKNOWN* and *UNKNOWN* of the *UNKNOWN* *UNKNOWN* *UNKNOWN* , maps *UNKNOWN*

** 11 **


DefaultDict{Int64,Array{Int64,1},Function} with 16 entries:
  2  => [16,17,18,41,51,133,146,159]
  11 => [13,14,28,60,61,67,72,75,76,78,136]
  0  => [1,2]
  7  => [7,35,42,45,57,58,157,160]
  9  => [19,53,55]
  10 => [20,62,66,77,137,154,155]
  8  => [26,50,63,82,132]
  6  => [4,25,37,39,43,48,54,56,59,69,138,142]
  4  => [6,15,30,31,65,71,139,140,149]
  3  => [21,24,46,131,141,145,148]
  5  => [5,10,11,23,27,32,34,40,49,52,74,79,96,97,109,123,124,128,129,135,152,156]
  14 => [8,44,87,91,93,95,100,110,111,112,113,114,115,116,117,119,120,121,122,126,127,130]
  13 => [70,80,83,85,86,88,89,90,92,94,99,101,102,104,105,106,107,108,118,125,158]
  15 => [29,143,144,150,153]
  12 => [84,98,103]
  1  => [3,9,12,22,33,36,38,47,64,68,73,81,134,147,151]

In [13]:
using Pipe
push!(LOAD_PATH, "../word-embeddings2")
using WordEmbeddings
we = @pipe load_embeddings("../word-embeddings2/word_emb_data/embeddings-scaled.EMBEDDING_SIZE=50.txt") |> WE(_...);



In [121]:
@pyimport hmmlearn.hmm as hl_hmm
@pyimport warnings
warnings.filterwarnings("ignore")

-303.7456690475301

In [270]:
hmm = hl_hmm.GaussianHMM(n_components=2, covariance_type="full", n_iter=200*1000*1000)

PyObject GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
      covars_weight=1,
      init_params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      means_prior=0, means_weight=0, n_components=2, n_iter=200000000,
      params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      random_state=None, startprob=None, startprob_prior=None, thresh=0.01,
      transmat=None, transmat_prior=None)

In [271]:
hmm[:fit](Any[ps])


PyObject GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
      covars_weight=1,
      init_params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      means_prior=0, means_weight=0, n_components=2, n_iter=200000000,
      params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      random_state=None, startprob=None, startprob_prior=1.0, thresh=0.01,
      transmat=None, transmat_prior=1.0)

In [272]:
hmm[:monitor_][:history] |> collect |> println
hmm[:monitor_][:n_iter]  |> println
for gmm in hmm[:gmms_]
    println(gmm[:converged_])   
end

{44663.90564653185,44663.905646725856}
200000000


LoadError: key not found: "gmms_"
while loading In[272], in expression starting on line 3

In [273]:
ENV["COLUMNS"]=100
labels = hmm[:predict](ps)
clusters = show_clusters(phrases, labels)

** 0 **
 - *UNKNOWN* *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* *UNKNOWN* *UNKNOWN*
 - family *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* *UNKNOWN* *UNKNOWN*
 - family *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* *UNKNOWN*
 - family *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* *UNKNOWN*
 - family *UNKNOWN* *UNKNOWN*
 - family *UNKNOWN* *UNKNOWN* *UNKNOWN* *UNKNOWN* turtles *UNKNOWN*
 - family *UNKNOWN* *UNKNOWN* *UNKNOWN* *UNKNOWN* and *UNKNOWN* *UNKNOWN* river turtles *UNKNOWN*
 - family *UNKNOWN*
 - *UNKNOWN* *UNKNOWN*
 - family *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* and *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* *UNKNOWN* *UNKNOWN*
 - family *UNKNOWN* *UNKNOWN* snapping turtles *UNKNOWN*
 - family *UNKNOWN* *UNKNOWN*
 - family *UNKNOWN* *UNKNOWN*
 - family *UNKNOWN* *UNKNOWN*
 - family *UNKNOWN* *UNKNOWN*
 - *UNKNOWN* *UNKNOW

DefaultDict{Int64,Array{Int64,1},Function} with 2 entries:
  0 => [85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,104,105,106,107,108,109,110,111,11…
  1 => [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,3…

In [266]:
wes, states = hmm[:sample](12)
bests = show_bests(we, wes',1);
@pipe bests[1,1:2:end][:] |> join(_, ' ')

LoadError: DimensionMismatch("dot product arguments have lengths 100 and 50")
while loading In[266], in expression starting on line 2